# Langfuseを使用したエージェントの評価

このクックブックでは、**[OpenAI agent SDK](https://github.com/openai/openai-agents-python)の内部ステップ（トレース）を監視**し、[Langfuse](https://langfuse.com/docs)を使用して**そのパフォーマンスを評価**する方法を学びます。

このガイドでは、チームがエージェントを迅速かつ確実に本番環境に導入するために使用される**オンライン**および**オフライン評価**メトリクスについて説明します。評価戦略についてさらに詳しく学ぶには、この[ブログ記事](https://langfuse.com/blog/2025-03-04-llm-evaluation-101-best-practices-and-challenges)をご確認ください。

**AIエージェント評価が重要な理由：**
- タスクが失敗したり最適でない結果を生成したりする際の問題のデバッグ
- コストとパフォーマンスのリアルタイム監視
- 継続的なフィードバックによる信頼性と安全性の向上

<br>

このテキストはHTMLコードのみで構成されており、翻訳すべき日本語のテキスト内容が含まれていません。これは動画を埋め込むためのiframeタグとCSSスタイリングのコードです。

HTMLコードやCSSプロパティは技術的な要素であり、翻訳の対象外となります。もし翻訳が必要な説明文やコメントなどのテキストコンテンツがございましたら、それを含めて再度お送りください。

## ステップ 0: 必要なライブラリのインストール

以下では、`openai-agents` ライブラリ（[OpenAI Agents SDK](https://github.com/openai/openai-agents-python)）、`pydantic-ai[logfire]` OpenTelemetry インストルメンテーション、`langfuse`、および Hugging Face の `datasets` ライブラリをインストールします。

In [ ]:
%pip install openai-agents nest_asyncio "pydantic-ai[logfire]" langfuse datasets

## ステップ1: エージェントを計測する

このノートブックでは、[Langfuse](https://langfuse.com/)を使用してエージェントのトレース、デバッグ、評価を行います。

**注意:** LlamaIndexやLangGraphを使用している場合は、それらの計測に関するドキュメントを[こちら](https://langfuse.com/docs/integrations/llama-index/workflows)と[こちら](https://langfuse.com/docs/integrations/langchain/example-python-langgraph)で確認できます。

In [2]:
import os
import base64

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

# Build Basic Auth header.
LANGFUSE_AUTH = base64.b64encode(
    f"{os.environ.get('LANGFUSE_PUBLIC_KEY')}:{os.environ.get('LANGFUSE_SECRET_KEY')}".encode()
).decode()
 
# Configure OpenTelemetry endpoint & headers
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = os.environ.get("LANGFUSE_HOST") + "/api/public/otel"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

# Your openai key
os.environ["OPENAI_API_KEY"] = "sk-proj-..."

環境変数が設定されたので、Langfuseクライアントを初期化できます。`get_client()`は、環境変数で提供された認証情報を使用してLangfuseクライアントを初期化します。

In [ ]:
from langfuse import get_client
 
langfuse = get_client()
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Pydantic Logfireは、OpenAi Agent SDKのインストルメンテーションを提供しています。これを使用して、[Langfuse OpenTelemetryバックエンド](https://langfuse.com/docs/opentelemetry/get-started)にトレースを送信します。

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import logfire

# Configure logfire instrumentation.
logfire.configure(
    service_name='my_agent_service',

    send_to_logfire=False,
)
# This method automatically patches the OpenAI Agents SDK to send logs via OTLP to Langfuse.
logfire.instrument_openai_agents()

## ステップ 2: インストルメンテーションをテストする

以下は簡単なQ&Aエージェントです。インストルメンテーションが正しく動作していることを確認するために実行します。すべてが正しく設定されている場合、観測可能性ダッシュボードにログ/スパンが表示されます。

In [8]:
import asyncio
from agents import Agent, Runner

async def main():
    agent = Agent(
        name="Assistant",
        instructions="You are a senior software engineer",
    )

    result = await Runner.run(agent, "Tell me why it is important to evaluate AI agents.")
    print(result.final_output)

loop = asyncio.get_running_loop()
await loop.create_task(main())

langfuse.flush()

13:00:52.784 OpenAI Agents trace: Agent workflow
13:00:52.787   Agent run: 'Assistant'
13:00:52.797     Responses API with 'gpt-4o'
Evaluating AI agents is crucial for several reasons:

1. **Performance Assessment**: It helps determine if the agent meets the desired goals and performs tasks effectively. By evaluating, we can assess accuracy, speed, and overall performance.

2. **Reliability and Consistency**: Regular evaluation ensures that the AI behaves consistently under different conditions and is reliable in production environments.

3. **Bias and Fairness**: Identifying and mitigating biases is essential for fair and ethical AI. Evaluation helps uncover any discriminatory patterns in the agent's behavior.

4. **Safety**: Evaluating AI agents ensures they operate safely and do not cause harm or unintended side effects, especially in critical applications.

5. **User Trust**: Proper evaluation builds trust with users and stakeholders by demonstrating that the AI is effective and al

[Langfuse Traces Dashboard](https://cloud.langfuse.com/traces)を確認して、スパンとログが記録されていることを確認してください。

Langfuseでのトレースの例：

![Langfuseでのトレースの例](https://langfuse.com/images/cookbook/integration_openai-agents/first-example-trace.png)

_[トレースへのリンク](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/0195948781a9f0d78fd5e067154aa508?timestamp=2025-03-14T12%3A01%3A03.401Z&observation=64bcac3cb82d04e9)_

## ステップ3: より複雑なエージェントの観察と評価

インストルメンテーションが動作することを確認したので、より複雑なクエリを試して、高度なメトリクス（トークン使用量、レイテンシ、コストなど）がどのように追跡されるかを見てみましょう。

In [9]:
import asyncio
from agents import Agent, Runner, function_tool

# Example function tool.
@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny."

agent = Agent(
    name="Hello world",
    instructions="You are a helpful agent.",
    tools=[get_weather],
)

async def main():
    result = await Runner.run(agent, input="What's the weather in Berlin?")
    print(result.final_output)

loop = asyncio.get_running_loop()
await loop.create_task(main())

13:01:15.351 OpenAI Agents trace: Agent workflow
13:01:15.355   Agent run: 'Hello world'
13:01:15.364     Responses API with 'gpt-4o'
13:01:15.999     Function: get_weather
13:01:16.000     Responses API with 'gpt-4o'
The weather in Berlin is currently sunny.


### トレース構造

Langfuseは**トレース**を記録し、エージェントのロジックの各ステップを表す**スパン**を含みます。ここでは、トレースには全体的なエージェント実行と以下のサブスパンが含まれています：
- ツール呼び出し（get_weather）
- LLM呼び出し（'gpt-4o'を使用したResponses API）

これらを調査することで、時間がどこで費やされているか、どれだけのトークンが使用されているかなどを正確に確認できます：

![Langfuseのトレースツリー](https://langfuse.com/images/cookbook/integration_openai-agents/trace-tree.png)

_[トレースへのリンク](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/019594b5b9a27c5d497b13be71e7f255?timestamp=2025-03-14T12%3A51%3A32.386Z&display=preview&observation=6374a3c96baf831d)_

## オンライン評価

オンライン評価とは、ライブな実世界環境、つまり本番環境での実際の使用中にエージェントを評価することを指します。これには、実際のユーザーインタラクションでのエージェントのパフォーマンスを監視し、結果を継続的に分析することが含まれます。

さまざまな評価技術に関するガイドを[こちら](https://langfuse.com/blog/2025-03-04-llm-evaluation-101-best-practices-and-challenges)に記載しています。

### 本番環境で追跡すべき一般的なメトリクス

1. **コスト** — 計装によってトークン使用量が取得され、トークンあたりの価格を割り当てることで概算コストに変換できます。
2. **レイテンシ** — 各ステップまたは全体の実行にかかる時間を観測します。
3. **ユーザーフィードバック** — ユーザーは直接的なフィードバック（いいね/だめ）を提供して、エージェントの改良や修正に役立てることができます。
4. **LLM-as-a-Judge** — 別のLLMを使用して、エージェントの出力をほぼリアルタイムで評価します（例：毒性や正確性のチェック）。

以下では、これらのメトリクスの例を示します。

#### 1. コスト

以下は`gpt-4o`呼び出しの使用量を示すスクリーンショットです。これは、コストの高いステップを確認し、エージェントを最適化するのに役立ちます。

![Costs](https://langfuse.com/images/cookbook/integration_openai-agents/gpt-4o-costs.png)

_[トレースへのリンク](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/019594b5b9a27c5d497b13be71e7f255?timestamp=2025-03-14T12%3A51%3A32.386Z&display=preview&observation=6374a3c96baf831d)_

#### 2. レイテンシ

各ステップの完了にかかった時間も確認できます。以下の例では、実行全体に7秒かかっており、これをステップごとに分解して確認できます。これにより、ボトルネックを特定し、エージェントを最適化することができます。

![Latency](https://langfuse.com/images/cookbook/integration_openai-agents/openai-agent-latency.png)

_[トレースへのリンク](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/019594b5b9a27c5d497b13be71e7f255?timestamp=2025-03-14T12%3A51%3A32.386Z&display=timeline&observation=b12967a01b3f8bcb)_

#### 3. 追加属性

Langfuseでは、スパンに追加属性を渡すことができます。これらには`user_id`、`tags`、`session_id`、およびカスタム`metadata`が含まれます。これらの詳細でトレースを充実させることは、異なるユーザーやセッション間でのアプリケーションの動作の分析、デバッグ、監視にとって重要です。

この例では、[user_id](https://langfuse.com/docs/tracing-features/users)、[session_id](https://langfuse.com/docs/tracing-features/sessions)、[trace_tags](https://langfuse.com/docs/tracing-features/tags)をLangfuseに渡しています。

In [10]:
input_query = "Why is AI agent evaluation important?"

with langfuse.start_as_current_span(
    name="OpenAI-Agent-Trace",
    ) as span:
    
    # Run your application here
    async def main(input_query):
            agent = Agent(
                name = "Assistant",
                instructions = "You are a helpful assistant.",
            )

            result = await Runner.run(agent, input_query)
            print(result.final_output)
            return result

    result = await main(input_query)
 
    # Pass additional attributes to the span
    span.update_trace(
        input=input_query,
        output=result,
        user_id="user_123",
        session_id="my-agent-session",
        tags=["staging", "demo", "OpenAI Agent SDK"],
        metadata={"email": "user@langfuse.com"},
        version="1.0.0"
        )
 
# Flush events in short-lived applications
langfuse.flush()

13:02:41.552 OpenAI Agents trace: Agent workflow
13:02:41.553   Agent run: 'Assistant'
13:02:41.554     Responses API with 'gpt-4o'
AI agent evaluation is crucial for several reasons:

1. **Performance Metrics**: It helps determine how well an AI agent performs its tasks, ensuring it meets the desired standards and objectives.

2. **Reliability and Safety**: Evaluation ensures the agent behaves consistently and safely in different scenarios, reducing risks of unintended consequences.

3. **Bias Detection**: By evaluating AI agents, developers can identify and mitigate biases, ensuring fair and equitable outcomes for all users.

4. **Benchmarking and Comparison**: Evaluation allows for the comparison of different AI models or versions, facilitating improvements and advancements.

5. **User Trust**: Demonstrating the effectiveness and reliability of an AI agent builds trust with users, encouraging adoption and usage.

6. **Regulatory Compliance**: Proper evaluation helps ensure AI system

![Langfuseでのトレース例](https://langfuse.com/images/cookbook/integration_openai-agents/openai-agent-sdk-custom-attributes.png)

#### 4. ユーザーフィードバック

エージェントがユーザーインターフェースに組み込まれている場合、直接的なユーザーフィードバック（チャットUIでの親指の上下のような）を記録できます。以下は、シンプルなフィードバック機能のために`IPython.display`を使用した例です。

下記のコードスニペットでは、ユーザーがチャットメッセージを送信する際に、OpenTelemetryのトレースIDを取得します。ユーザーが最後の回答を気に入った/気に入らなかった場合、そのトレースにスコアを付与します。

In [38]:
from agents import Agent, Runner, WebSearchTool
from opentelemetry.trace import format_trace_id
import ipywidgets as widgets
from IPython.display import display
from langfuse import get_client
 
langfuse = get_client()

# Define your agent with the web search tool
agent = Agent(
    name="WebSearchAgent",
    instructions="You are an agent that can search the web.",
    tools=[WebSearchTool()]
)

def on_feedback(button):
    if button.icon == "thumbs-up":
      langfuse.create_score(
            value=1,
            name="user-feedback",
            comment="The user gave this response a thumbs up",
            trace_id=trace_id
        )
    elif button.icon == "thumbs-down":
      langfuse.create_score(
            value=0,
            name="user-feedback",
            comment="The user gave this response a thumbs down",
            trace_id=trace_id
        )
    print("Scored the trace in Langfuse")

user_input = input("Enter your question: ")

# Run agent
with langfuse.start_as_current_span(
    name="OpenAI-Agent-Trace",
    ) as span:
    
    # Run your application here
    result = Runner.run_sync(agent, user_input)
    print(result.final_output)

    result = await main(user_input)
    trace_id = langfuse.get_current_trace_id()

    span.update_trace(
        input=user_input,
        output=result.final_output,
    )

# Get feedback
print("How did you like the agent response?")

thumbs_up = widgets.Button(description="👍", icon="thumbs-up")
thumbs_down = widgets.Button(description="👎", icon="thumbs-down")

thumbs_up.on_click(on_feedback)
thumbs_down.on_click(on_feedback)

display(widgets.HBox([thumbs_up, thumbs_down]))

# Flush events in short-lived applications
langfuse.flush()

Enter your question: What is Langfuse?
13:54:41.574 OpenAI Agents trace: Agent workflow
13:54:41.575   Agent run: 'WebSearchAgent'
13:54:41.577     Responses API with 'gpt-4o'
Langfuse is an open-source engineering platform designed to enhance the development, monitoring, and optimization of Large Language Model (LLM) applications. It offers a suite of tools that provide observability, prompt management, evaluations, and metrics, facilitating the debugging and improvement of LLM-based solutions. ([toolkitly.com](https://www.toolkitly.com/langfuse?utm_source=openai))

**Key Features of Langfuse:**

- **LLM Observability:** Langfuse enables developers to monitor and analyze the performance of language models by tracking API calls, user inputs, prompts, and outputs. This observability aids in understanding model behavior and identifying areas for improvement. ([toolkitly.com](https://www.toolkitly.com/langfuse?utm_source=openai))

- **Prompt Management:** The platform provides tools for m

Scored the trace in Langfuse


ユーザーフィードバックはLangfuseで取得されます：

![ユーザーフィードバックがLangfuseで取得されている様子](https://langfuse.com/images/cookbook/integration_openai-agents/open-ai-agent-user-feedback.png)

#### 5. LLM-as-a-Judge

LLM-as-a-Judgeは、エージェントの出力を自動的に評価するもう一つの方法です。別のLLM呼び出しを設定して、出力の正確性、毒性、スタイル、またはその他の気になる基準を評価することができます。

**ワークフロー**:
1. **評価テンプレート**を定義します。例：「テキストが毒性を含んでいるかチェックしてください。」
2. 判定モデルとして使用するモデルを設定します。この場合は`gpt-4o-mini`です。
2. エージェントが出力を生成するたびに、その出力をテンプレートと共に「判定」LLMに渡します。
3. 判定LLMは評価や ラベルで応答し、それを観測可能性ツールにログとして記録します。

Langfuseからの例：

![LLM-as-a-Judge評価テンプレート](https://langfuse.com/images/cookbook/integration_openai-agents/evaluator-template.png)
![LLM-as-a-Judge評価器](https://langfuse.com/images/cookbook/integration_openai-agents/evaluator.png)

In [40]:
# Example: Checking if the agent’s output is toxic or not.
from agents import Agent, Runner, WebSearchTool

# Define your agent with the web search tool
agent = Agent(
    name="WebSearchAgent",
    instructions="You are an agent that can search the web.",
    tools=[WebSearchTool()]
)

input_query = "Is eating carrots good for the eyes?"

# Run agent
with langfuse.start_as_current_span(name="OpenAI-Agent-Trace") as span:
    # Run your agent with a query
    result = Runner.run_sync(agent, input_query)

    # Add input and output values to parent trace
    span.update_trace(
        input=input_query,
        output=result.final_output,
    )

14:05:34.735 OpenAI Agents trace: Agent workflow
14:05:34.736   Agent run: 'WebSearchAgent'
14:05:34.738     Responses API with 'gpt-4o'


この例の回答は「有害ではない」と判定されていることがわかります。

![LLM-as-a-Judge評価スコア](https://langfuse.com/images/cookbook/integration_openai-agents/llm-as-a-judge-score.png)

#### 6. 観測可能性メトリクスの概要

これらのメトリクスはすべてダッシュボードで一緒に可視化できます。これにより、多くのセッションにわたってエージェントのパフォーマンスを素早く確認でき、品質メトリクスを時系列で追跡することができます。

![観測可能性メトリクスの概要](https://langfuse.com/images/cookbook/integration_openai-agents/dashboard-dark.png)

## オフライン評価

オンライン評価はライブフィードバックに不可欠ですが、**オフライン評価**も必要です。これは開発前または開発中の体系的なチェックです。これにより、変更を本番環境にロールアウトする前に品質と信頼性を維持できます。

### データセット評価

オフライン評価では、通常以下の手順を行います：
1. ベンチマークデータセット（プロンプトと期待される出力のペアを含む）を用意する
2. そのデータセット上でエージェントを実行する
3. 出力を期待される結果と比較するか、追加のスコアリング機能を使用する

以下では、[search-dataset](https://huggingface.co/datasets/junzhang1207/search-dataset)を使用したこのアプローチを実演します。このデータセットには、ウェブ検索ツールで回答可能な質問と期待される回答が含まれています。

In [44]:
import pandas as pd
from datasets import load_dataset

# Fetch search-dataset from Hugging Face
dataset = load_dataset("junzhang1207/search-dataset", split = "train")
df = pd.DataFrame(dataset)
print("First few rows of search-dataset:")
print(df.head())

README.md:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

data-samples.json:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

data.jsonl:   0%|          | 0.00/316k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/934 [00:00<?, ? examples/s]

First few rows of GSM8K dataset:
                                     id  \
0  20caf138-0c81-4ef9-be60-fe919e0d68d4   
1  1f37d9fd-1bcc-4f79-b004-bc0e1e944033   
2  76173a7f-d645-4e3e-8e0d-cca139e00ebe   
3  5f5ef4ca-91fe-4610-a8a9-e15b12e3c803   
4  64dbed0d-d91b-4acd-9a9c-0a7aa83115ec   

                                            question  \
0                 steve jobs statue location budapst   
1  Why is the Battle of Stalingrad considered a t...   
2  In what year did 'The Birth of a Nation' surpa...   
3  How many Russian soldiers surrendered to AFU i...   
4   What event led to the creation of Google Images?   

                                     expected_answer       category       area  
0  The Steve Jobs statue is located in Budapest, ...           Arts  Knowledge  
1  The Battle of Stalingrad is considered a turni...   General News       News  
2  This question is based on a false premise. 'Th...  Entertainment       News  
3  About 300 Russian soldiers surrendered to th

次に、実行を追跡するためにLangfuseでデータセットエンティティを作成します。その後、データセットから各アイテムをシステムに追加します。

In [52]:
from langfuse import get_client
langfuse = get_client()

langfuse_dataset_name = "search-dataset_huggingface_openai-agent"

# Create a dataset in Langfuse
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="search-dataset uploaded from Huggingface",
    metadata={
        "date": "2025-03-14",
        "type": "benchmark"
    }
)

Dataset(id='cm88w66t102qpad07xhgeyaej', name='search-dataset_huggingface_openai-agent', description='search-dataset uploaded from Huggingface', metadata={'date': '2025-03-14', 'type': 'benchmark'}, project_id='cloramnkj0002jz088vzn1ja4', created_at=datetime.datetime(2025, 3, 14, 14, 47, 14, 676000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 3, 14, 14, 47, 14, 676000, tzinfo=datetime.timezone.utc))

In [53]:
for idx, row in df.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["expected_answer"]}
    )
    if idx >= 49: # For this example, we upload only the first 50 items
        break

![Langfuseのデータセット項目](https://langfuse.com/images/cookbook/integration_openai-agents/example-dataset.png)

#### データセット上でのエージェントの実行

`run_openai_agent()`ヘルパー関数を定義します。この関数は以下を行います：
1. Langfuseスパンを開始
2. プロンプトに対してエージェントを実行
3. LangfuseでトレースIDを記録

その後、各データセット項目をループし、エージェントを実行して、トレースをデータセット項目にリンクします。必要に応じて、簡単な評価スコアを添付することもできます。

In [ ]:
from agents import Agent, Runner, WebSearchTool
from langfuse import get_client
 
langfuse = get_client()
dataset_name = "search-dataset_huggingface_openai-agent"
current_run_name = "qna_model_v3_run_05_20" # Identifies this specific evaluation run

agent = Agent(
    name="WebSearchAgent",
    instructions="You are an agent that can search the web.",
    tools=[WebSearchTool(search_context_size= "high")]
)
 
# Assume 'run_openai_agent' is your instrumented application function
def run_openai_agent(question):
    with langfuse.start_as_current_generation(name="qna-llm-call") as generation:
        # Simulate LLM call
        result = Runner.run_sync(agent, question)
 
        # Update the trace with the input and output
        generation.update_trace(
            input= question,
            output=result.final_output,
        )

        return result.final_output
 
dataset = langfuse.get_dataset(name=dataset_name) # Fetch your pre-populated dataset
 
for item in dataset.items:
 
    # Use the item.run() context manager
    with item.run(
        run_name=current_run_name,
        run_metadata={"model_provider": "OpenAI", "temperature_setting": 0.7},
        run_description="Evaluation run for Q&A model v3 on May 20th"
    ) as root_span: # root_span is the root span of the new trace for this item and run.
        # All subsequent langfuse operations within this block are part of this trace.
 
        # Call your application logic
        generated_answer = run_openai_agent(question=item.input["text"])

        print(item.input)

このプロセスを以下の異なる設定で繰り返すことができます：
- 検索ツール（例：OpenAIの`WebSearchTool`の異なるコンテキストサイズ）
- モデル（gpt-4o-mini、o1など）
- ツール（検索ありと検索なし）

その後、Langfuseで並べて比較できます。この例では、50個のデータセット質問に対してエージェントを3回実行しました。各実行では、OpenAIの`WebSearchTool`のコンテキストサイズに異なる設定を使用しました。コンテキストサイズを増やすことで、回答の正確性も`0.89`から`0.92`にわずかに向上したことがわかります。`correct_answer`スコアは、データセットで提供されたサンプル回答に基づいて質問の正確性を判定するように設定された[LLM-as-a-Judge Evaluator](https://langfuse.com/docs/scores/model-based-evals)によって作成されます。

![Dataset run overview](https://langfuse.com/images/cookbook/integration_openai-agents/dataset_runs.png)
![Dataset run comparison](https://langfuse.com/images/cookbook/integration_openai-agents/dataset-run-comparison.png)